In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.7.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia"  # CSV DataFrames Random Statistics JWAS
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.7.1 on the current Colab Runtime...
2022-07-23 05:16:51 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.7/julia-1.7.1-linux-x86_64.tar.gz [123374573/123374573] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package CUDA...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.7

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


In [ ]:
versioninfo()

Julia Version 1.7.1
Commit ac5cc99908 (2021-12-22 19:35 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Mixed Effects Neural Networks NNMM

# Part1. introduction

## 1. Overview
The Mixed Effect Neural Networks (NN-MM) extend linear mixed model ("MM") to multilayer neural networks ("NN") by adding one middle layer between genotype layer and phenotypes layer. Nodes in the middle layer represent intermediate traits, e.g., the known intermediate omics features such as gene expression levels can be incorporated in the middle layer. These three sequential layers form a unified network. 

![](https://github.com/zhaotianjing/figures/blob/main/omics_example.png?raw=true)

NN-MM allow any patterns of missing data in the middle layer, and missing data will be sampled. In below figure, for an individual, the gene expression levels of the first two genes are 0.9 and 0.1, respectively, and the gene expression level of the last gene is missing to be sampled. The missing patterns of gene expression levels can be different for different individuals.



## 2. Extend linear mixed model to multilayer neural networks

Multiple independent single-trait mixed models are used to model the relationships between input layer (genotypes) and middle layer (intermediate traits). Activation functions in the neural network are used to approximate the linear/nonlinear relationships between middle layer (intermediate traits) and output layer (phenotypes). Missing values in the middle layer (intermediate traits) are sampled by Hamiltonian Monte Carlo based on the upstream genotype layer and downstream phenotype layer.

Details can be found in our publications:

> * Tianjing Zhao, Jian Zeng, and Hao Cheng. Extend mixed models to multilayer neural networks for genomic prediction including intermediate omics data, GENETICS, 2022; [https://doi.org/10.1093/genetics/iyac034](https://doi.org/10.1093/genetics/iyac034). 
> * Tianjing Zhao, Rohan Fernando, and Hao Cheng. Interpretable artificial neural networks incorporating Bayesian alphabet models for genome-wide prediction and association studies, G3 Genes|Genomes|Genetics, 2021;  [https://doi.org/10.1093/g3journal/jkab228](https://doi.org/10.1093/g3journal/jkab228)

## 3. Flexibility

NN-MM can fit fully-connected neural networks ((a),(b)), or partial-connected neural networks ((c),(d)). Also, the relationship between middle layer (intermediate traits) and output layer (phenotypes) can be based on activation functions ((a),(c)), or pre-defined by a user-defined function ((b),(d)).

![](https://github.com/zhaotianjing/figures/blob/main/wiki_full_vs_partial.png?raw=true)

## 4. Multi-threaded parallelism

By default, multiple single-trait models will be used to model the relationships between input layer (genotypes) and middle layer (intermediate traits). Multi-threaded parallelism will be used for parallel computing. The number of threads can be checked by running `Threads.nthreads()` in Julia. Usually, using multiple threads will be about 3 times faster than using a single thread.

The number of execution threads is controlled by using the -t/--threads command-line argument (requires at least Julia 1.5). 

For example, to start Julia with 4 threads:

```{julia}
julia --threads 4
```

If you're using Juno via IDE like Atom, all threads will be loaded automatically. 

# Part 2: Mixed effect neural network: Genotypes -> Unobserved intemediate traits -> Phenotyes

Tips: please center the phenotypes to have zero mean.

## Example(a): fully-connected neural networks, all intemediate traits are unobserved
- nonlinear function (to define relationship between middle layer and phenotye): tanh (other supported activation functions: "sigmoid", "relu", "leakyrelu", "linear")
- number of nodes in the middle layer: 3
- Bayesian model: multiple independent single-trait BayesC (to sample marker effects on intemediate traits). Note, to use multi-trait Bayesian Alphabet models, please set `mega_trait=false` in `runMCMC()` function.
- sample the unobserved intemediate traits in the middle layer: Hamiltonian Monte Carlo

![](https://github.com/zhaotianjing/figures/blob/main/part2_example.png?raw=true)

In [ ]:
# Step 1: Load packages
using JWAS,DataFrames,CSV,Statistics,JWAS.Datasets,Random
Random.seed!(123)

# Step 2: Read data 
phenofile  = dataset("phenotypes.csv") #get example data path
genofile   = dataset("genotypes.csv")  #get example data path

phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstrings=["NA"]) #read phenotypes (output layer)
genotypes  = get_genotypes(genofile,separator=',',method="BayesC");                      #read genotypes  (input layer)


# Step 3: Build Model Equations 
model_equation  ="y1 = intercept + genotypes"  #name of phenotypes is "y1" in the phenotypes data
                                               #name of genotypes is "genotypes" (user-defined in the previous step)
                                               #the single-trait mixed model used between input and each node in middle layer is: middle node = intercept + genotypes
model = build_model(model_equation,
		    num_hidden_nodes=3,            #number of nodes in middle layer is 3
		    nonlinear_function="tanh");    #tanh function is used to approximate relationship between middle layer and phenotype


# Step 4: Run Analysis
out=runMCMC(model,phenotypes,chain_length=5000); 

# Step 5: Check Accuruacy
results    = innerjoin(out["EBV_NonLinear"], phenotypes, on = :ID) 
accuruacy  = cor(results[!,:EBV],results[!,:bv1])

## Example output files

The i-th middle nodes will be named as "trait name"+"i". In our example, the observed trait is named "y1", and there are 3 middle nodes, so the middle nodes are named as "y11", "y12", and "y13", respectively.

Below is a list of files containing estimates and standard deviations for variables of interest. 

| file name      | description |
| -----------    | ----------- |
| EBV_NonLinear.txt | estimated breeding values for observed trait  |
| EBV_y11.txt       | estimated breeding values for middle node 1      |
| EBV_y12.txt       | estimated breeding values for middle node 2      |
| EBV_y13.txt       | estimated breeding values for middle node 3      |
|genetic_variance.txt                   | estimated genetic variance-covariance of all middle nodes |
|heritability.txt                       | estimated heritability of all middle nodes                |
|location_parameters.txt                | estimated bias of all middle nodes                        |
|neural_networks_bias_and_weights.txt.  | estimated bias of phenotypes and weights between middle nodes and phenotypes|
|pi_genotypes.txt                       | estimated pi of all middle nodes                          | 
|marker_effects_genotypes.txt           | estimated marker effects of all middle nodes              |
|residual_variance.txt                  | estimated residual variance-covariance for all middle nodes| 

Below is a list of files containing MCMC samples for variables of interest. 

| file name      | description |
| -----------    | ----------- |
| MCMC_samples_EBV_NonLinear.txt     | MCMC samples from the posterior distribution of breeding values for phenotypes  |
| MCMC_samples_EBV_y11.txt     | MCMC samples from the posterior distribution of breeding values for middle node 1       |
| MCMC_samples_EBV_y12.txt     | MCMC samples from the posterior distribution of breeding values for middle node 2       |
| MCMC_samples_EBV_y13.txt     | MCMC samples from the posterior distribution of breeding values for middle node 3       |
|MCMC_samples_genetic_variance.txt                   | MCMC samples from the posterior distribution of genetic variance-covariance for all middle nodes   |
|MCMC_samples_heritability.txt                       | MCMC samples from the posterior distribution of heritability for all middle nodes                                | 
|MCMC_samples_marker_effects_genotypes_y11            |MCMC samples from the posterior distribution of marker effects for middle node 1           |
|MCMC_samples_marker_effects_genotypes_y12            |MCMC samples from the posterior distribution of marker effects for middle node 2           |
|MCMC_samples_marker_effects_genotypes_y13            |MCMC samples from the posterior distribution of marker effects for middle node 3           |
|MCMC_samples_marker_effects_variances_genotypes.txt | MCMC samples from the posterior distribution of marker effect variance for all middle nodes        |
|MCMC_samples_neural_networks_bias_and_weights.txt.  | MCMC samples from the posterior distribution of bias of observed trait and weights between middle nodes and phenotypes|
|MCMC_samples_pi_genotypes.txt                       | MCMC samples from the posterior distribution of pi for all middle nodes                                          |
|MCMC_samples_residual_variance.txt                  |  MCMC samples from the posterior distribution of residual variance-covariance for all middle nodes |